In [28]:
import pandas as pd


In [33]:
datadir = "../MSMS/"
polarities = ["pos/","neg/"]
fn = "filtered_results_combined.csv"
fn_annotation = {"pos/":"Copy of putative_IDS_transition_list_MSH_v4.csv","neg/":"putative_IDS_transition_list_MSH_v2.csv"}
mapper = {"pos/":"POS","neg/":"NEG"}
mzMapper = {"pos/":1.0073,"neg/":-1.0073}

In [35]:
for pol in polarities:
    data = pd.read_csv(datadir + pol + fn)
    annotation = pd.read_csv(datadir + pol + fn_annotation[pol])
    for column in ["Tier MSH","Comment MSH","Curated name"]:
        data[column] = annotation[column]
    data = data[data["Tier MSH"] < 5]
    toAdd = pd.read_csv(datadir + "IROA_presence_in_plasma.csv")
    toAdd = toAdd[toAdd["NEG or POS"] == mapper[pol]]
    toDrop = [x for x in toAdd.index.values if toAdd.at[x,"CAS ID"] in data["DB_Compound_ID"]]
    toAdd = toAdd.drop(toDrop)
    newDf = pd.DataFrame()
    newDf["Tier MSH"] = [1 for _ in range(len(toAdd))]
    newDf["Comment MSH"] = ["added from IROA" for _ in range(len(toAdd))]
    newDf["Curated name"] = [row["Compound Name"] for _,row in toAdd.iterrows()]
    newDf["found in other polarity"] = ['no' for _ in range(len(toAdd))]
    newDf["compound_m/z"] = [row["Mass"] + mzMapper[pol] for _,row in toAdd.iterrows()]
    newDf["rt"] = toAdd["RT"].values
    newDf["compound_rt"] = toAdd["RT"].values
    newDf["compound_formula"] = [row["Formula"].replace(" ","") for _,row in toAdd.iterrows()]
    newDf["DB_Compound_ID"] = toAdd["CAS ID"].values
    newDf["Compound_Name"] = toAdd["Compound Name"].values
    newDf["#featureID"] = [-1 for _ in range(len(toAdd))]
    for x in ["#featureID","isolation_center_m/z","DB_Spectrum_ID","dot_product","ppm_Error","isotope_similiarity","Abundance"]:
        newDf[x] = [-1 for _ in range(len(toAdd))]
    for x in ["redundant","Origin File","Compound Matched to Multiple Feats","same ID in other polarity"]:
        newDf[x] = ["" for _ in range(len(toAdd))]
    data = pd.concat((data,newDf),axis=0,ignore_index=True)
    data.to_csv(datadir + pol + "filtered_results_combined_with_annotation_added_IROA.csv",index=False)
    print(data.shape)

(176, 22)
(257, 22)
